In [ ]:
%%capture

import warnings

warnings.filterwarnings("ignore")


import altair as alt
import calitp_data_analysis.magics
import deploy_portfolio_yaml
import pandas as pd

from IPython.display import HTML, Image, Markdown, display, display_html
from omegaconf import OmegaConf
from update_vars import GTFS_DATA_DICT, RT_SCHED_GCS, SCHED_GCS, SEGMENT_GCS

readable_dict = OmegaConf.load("readable2.yml")

import viz_data_prep
import _report_visuals_utils
import aggregate_filter_operator_data
import _report_operator_visuals

In [ ]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
# Comment out and leave this cell right below pandas
portfolio_name = "City and County of San Francisco"
# portfolio_name = "City of West Hollywood"

In [ ]:
from IPython.display import HTML, display

# Set drop down menu to be on the upper right for the charts
display(
    HTML(
        """
<style>
form.vega-bindings {
  position: absolute;
  right: 0px;
  top: 0px;
}
</style>
"""
    )
)

In [ ]:
# %%capture_parameters
# portfolio_name

In [ ]:
ROUTE_DIR_FILE = GTFS_DATA_DICT.digest_tables.route_schedule_vp

# some of the portfolio grain can be dealt with
# but separate out the renaming/replacing/subsetting to separate script

route_dir_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{ROUTE_DIR_FILE}.parquet",
    filters=[
        [
            ("portfolio_organization_name", "==", portfolio_name),
        ]
    ],
).pipe(
    viz_data_prep.data_wrangling_for_visualizing,
    viz_data_prep.route_direction_cols_for_viz,
    viz_data_prep.readable_col_names,
)

In [ ]:
route_df = aggregate_filter_operator_data.grab_most_recent_geography(portfolio_name)

In [ ]:
OPERATOR_PROFILE = GTFS_DATA_DICT.digest_tables.operator_profiles
profile_df = pd.read_parquet(
    f"{RT_SCHED_GCS}{OPERATOR_PROFILE}_recent.parquet",
    filters=[
        [
            ("portfolio_organization_name", "==", portfolio_name),
        ]
    ],
)

# {organization_name}

## Operator Overview

In [ ]:
try:
    date = profile_df["service_date"].dt.to_pydatetime()[0]
except:
    pass

In [ ]:
service_area = int(profile_df.service_area_sq_miles.values[0])
service_area = "{:,}".format(service_area)
service_pop = int(profile_df.service_area_pop.values[0])
service_pop = "{:,}".format(service_pop)

In [ ]:
display(
    Markdown(
        f"""{portfolio_name} is headquartered in <b>{profile_df.hq_city.values[0]}</b> in the Urbanized Area of <b>{profile_df.primary_uza_name.values[0]}</b>.<br>
            This operator provides <b>{service_area}</b> square miles of public transit service, which has a service population of <b>{service_pop}</b>.<br>
            This organization is a {profile_df.reporter_type.values[0]}.<br>
            <b>Data Source</b>: <a href="https://www.transit.dot.gov/ntd/data-product/2022-annual-database-agency-information">National Transit Database</a> Annual Agency Information.
            """
    )
)

### Route Typologies

In [ ]:
try:
    display(
        Markdown(
            f"""The following data presents an overview of GTFS characteristics using data from the most recent date of
      <b>{date.strftime("%B")} {date.year}</b>."""
        )
    )
except:
    pass

In [ ]:
try:
    n_routes = profile_df["operator_n_routes"].values[0]
    n_routes = "{:,}".format(n_routes)
    display(
        Markdown(
            f"""{portfolio_name} runs <b>{n_routes}</b> unique routes. Below is the breakdown of the routes and routes can belong to one or more categories.<p>
            Route categories are determined using a approach that looks at GTFS trips data
        alongside National Association of City Transportation Officials (NACTO)'s
        <a href="https://nacto.org/publication/transit-street-design-guide/introduction/service-context/transit-route-types/">Transit Route Types</a> 
        and <a href= "https://nacto.org/publication/transit-street-design-guide/introduction/service-context/transit-frequency-volume/">Frequency and Volume</a>
        guides. Please see the <a href="https://github.com/cal-itp/data-analyses/blob/main/gtfs_digest/methodology.md">methodology docs</a> for more details on this approach.
        """
        )
    )
except:
    display(Markdown(f"""{portfolio_name} doesn't have an operator profile."""))

In [ ]:
try:
    display(_report_operator_visuals.route_typology_chart(profile_df))
except:
    display(
        Markdown(f"""{portfolio_name} doesn't have route typology data available.""")
    )

### Service Area

In [ ]:
try:
    n_service_miles = int(profile_df.operator_route_length_miles.values[0])
    n_service_miles = "{:,}".format(n_service_miles)
    display(
        Markdown(
            f"""{portfolio_name} provides <b>{n_service_miles}</b> miles of public transit.
        """
        )
    )

except:
    pass

In [ ]:
try:
    display(_report_operator_visuals.percentile_routes_chart(route_df))
except:
    pass

In [ ]:
try:
    display(
        route_df[["Route", "Route Length (Miles)", "geometry"]].explore(
            "Route",
            cmap="Spectral",
            tiles="CartoDB positron",
            width=500,
            height=300,
            style_kwds={"weight": 3},
            legend=False,
            tooltip=["Route", "Route Length (Miles)"],
        )
    )
except:
    display(Markdown(f"""{portfolio_name} doesn't have an route geographies."""))

In [ ]:
try:
    display(
        Markdown(
            f"{portfolio_name} provides public transit in {profile_df['counties_served'].values[0]} counties."
        )
    )
except:
    display(Markdown(f"""{portfolio_name} doesn't have an route geographies."""))

In [ ]:
try:
    n_stops = int(profile_df["operator_n_stops"].values[0])
    n_stops = "{:,}".format(n_stops)

    n_arrivals = int(profile_df["operator_n_arrivals"].values[0])
    n_arrivals = "{:,}".format(n_arrivals)
    display(
        Markdown(
            f"""{portfolio_name} stops at <b>{n_stops}</b> unique stops, 
                totaling to <b>{n_arrivals}</b> arrivals.
            """
        )
    )
except:
    pass

In [ ]:
try:
    display(_report_visuals_utils.route_filter(route_dir_df))
except:
    display(Markdown(f"""{portfolio_name} only has schedule data."""))